## knock 055 数値からカテゴリデータを作成する

In [3]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列が省略されないようにする

polars.config.Config

### データを読み込む

In [4]:
df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
display(df_receipt.head())

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


### ノック

In [6]:
# 顧客ID毎の売上高
df_group_by = (
    df_receipt
    .group_by(by = "customer_id")
    .agg( pl.sum("amount").alias("sum_amount") )
)

# 四分位数の計算
pct25 = df_group_by["sum_amount"].quantile(quantile = 0.25)
pct50 = df_group_by["sum_amount"].quantile(quantile = 0.5)
pct75 = df_group_by["sum_amount"].quantile(quantile = 0.75)

# カテゴリに分ける
def pct_group(x):
    if x < pct25:
        return 1
    elif x < pct50:
        return 2
    elif x < pct75:
        return 3
    else:
        return 4


(
    df_group_by
    # 売上高のカテゴリ列を加える
    .with_columns(
        pl.col("sum_amount").map_elements(lambda x: pct_group(x)).alias("pct_group")
    )
    .head(n = 10)
)

customer_id,sum_amount,pct_group
str,i64,i64
"""CS034415000122…",4982,4
"""CS007415000037…",316,1
"""CS040215000047…",3571,3
"""CS032215000073…",5520,4
"""CS001515000633…",744,2
"""CS011414000128…",3781,4
"""CS002514000240…",626,2
"""CS037513000193…",404,1
"""CS030315000042…",275,1
